In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/dumping/clean/dumping-parliament.txt

In [3]:
# import malaya
# model = malaya.segmentation.transformer('super-tiny-t5', use_huggingface = True)
# model.greedy_decoder(['TUAN LIM LIP ENG [ SEGAM BUT ] m inta M ENTERI IMalaysia ( BRIM ) bermula daripada projek tersebut dilaksanakan mengikut kategori penerima dan tahun .'])

In [5]:
rejected_prefix = ['JAWAPAN-JAWAPAN', 'NOTA', 'NO SOALAN', 'PERTANYAAN DEWAN', 'Sekian', 'Terima kasih',
                  'JAWAPAN']

In [6]:
def validate_spacing_chars(string, min_threshold = 0.2):
    splitted = string.split()
    single_chars = [s for s in splitted if len(s) == 1]
    if len(single_chars):
        return len(single_chars) / len(splitted) > min_threshold
    else:
        return False

In [20]:
import re
re_roman = r"^M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$".lower()
VOWELS = set('aeiou')

def check_roman(string):
    splitted = string.split()
    for s in splitted:
        if bool(re.search(re_roman,s)):
            return True
    return False

In [21]:
with open('dumping-parliament.txt') as fopen:
    parliament = fopen.read().split('\n')
parliament = [t for t in parliament if all([not t.lower().startswith(p.lower()) for p in rejected_prefix])]
parliament = [t for t in parliament if not t.isupper() and not validate_spacing_chars(t)]
parliament = [t for t in parliament if not check_roman(t)]
parliament = [t for t in parliament if len(set(t) & VOWELS)]
parliament = list(filter(None, parliament))
len(parliament)

122692

In [22]:
from tqdm import tqdm

min_len = 2
max_len = 25

selected_parliament = []
for t in tqdm(parliament):
    l = len(t.split())
    if min_len < l < max_len:
        selected_parliament.append(t.replace('/ o', ''))

100%|██████████| 122692/122692 [00:00<00:00, 1012953.14it/s]


In [28]:
selected_parliament = list(set(selected_parliament))

In [27]:
import json
from unidecode import unidecode

def put_spacing_num(string):
    string = re.sub('[A-Za-z]+', lambda ele: ' ' + ele[0] + ' ', string)
    return re.sub(r'[ ]+', ' ', string).strip()

def convert_to_ascii(string):
    return unidecode(string)

def collapse_whitespace(string):
    return re.sub(_whitespace_re, ' ', string)

def cleaning(string, normalize = True, add_eos = False):
    sequence = []
    string = convert_to_ascii(string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    if string[-1] in ['-', ',']:
        string = string[:-1]
    if string[-1] != '.':
        string = string + '.'
    string = put_spacing_num(string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = string
    return string

In [29]:
samples = [{'text': t, 'cleaned': cleaning(t)} for t in selected_parliament]

In [30]:
samples[:10]

[{'text': 'PERDAGANGAN ANTARABANGSA DAN INDUSTRI menyatakan usaha memastikan pihak industri boleh mendapat para pekerja asing dengan kos munasabah tanpa dieksploitasikan dan birokrasi .',
  'cleaned': 'PERDAGANGAN ANTARABANGSA DAN INDUSTRI menyatakan usaha memastikan pihak industri boleh mendapat para pekerja asing dengan kos munasabah tanpa dieksploitasikan dan birokrasi .'},
 {'text': 'Sehubungan itu , sistem pemeriksaan pasport sedia ada perlu dikonfigurasi untuk disesuaikan dengan ciri-ciri baru PMA .',
  'cleaned': 'Sehubungan itu , sistem pemeriksaan pasport sedia ada perlu dikonfigurasi untuk disesuaikan dengan ciri - ciri baru PMA .'},
 {'text': 'Lampiran I Projek perkilangan yang dilaksanakan mengikut negeri bagi.',
  'cleaned': 'Lampiran I Projek perkilangan yang dilaksanakan mengikut negeri bagi .'},
 {'text': 'Projek ini akan dilaksanakan secara berperingkat-peringkat dalam setiap Rancangan 5 Tahun dan tertakluk kepada kemampuan kewangan Kerajaan .',
  'cleaned': 'Projek in

In [31]:
with open('populated-parliament.json', 'w') as fopen:
    json.dump(samples, fopen)